# Example stage reports

Example stage reports based on comparing current and previous stage.

In [12]:
import re
match = re.fullmatch(r"^([^a]+)(a+)(a)$", "bbbaaa")
match = re.fullmatch(r"^([^a]+)(a+)$", "bbbaaa")
match = re.fullmatch(r"^([^abc]+)([abc]+)$", "ddeeedddd")
print(match.group(0))

AttributeError: 'NoneType' object has no attribute 'group'

In [8]:
import pandas as pd
from itertools import groupby


def detect_tussles_in_dataframe(df, min_changes=2, max_run_length=1, max_gap=1, min_lead_changes=2):
    results = []

    # Get unique drivers and their position strings
    drivers = df[["driverName", "posEndingString"]].values

    for i in range(len(drivers)):
        driver_a_name, driver_a_pos = drivers[i]
        for j in range(i + 1, len(drivers)):
            driver_b_name, driver_b_pos = drivers[j]

            min_len = min(len(driver_a_pos), len(driver_b_pos))
            a_positions = driver_a_pos[:min_len]
            b_positions = driver_b_pos[:min_len]

            relative_positions = []
            relative_leads = []

            for k in range(min_len):
                pos_a = a_positions[k]
                pos_b = b_positions[k]

                pos_a_num = ord(pos_a) - ord("a") + 1
                pos_b_num = ord(pos_b) - ord("a") + 1

                if pos_a_num < pos_b_num:
                    lead = "A"
                elif pos_b_num < pos_a_num:
                    lead = "B"
                else:
                    lead = "same"

                relative_positions.append((k, pos_a, pos_b, lead))
                relative_leads.append(lead)

            tussle_segments = []
            current_segment = None

            for idx, (pos_idx, pos_a, pos_b, lead) in enumerate(relative_positions):
                if current_segment is None and lead != "same":
                    current_segment = {
                        "start": pos_idx,
                        "positions": [(pos_a, pos_b)],
                        "a_pattern": pos_a,
                        "b_pattern": pos_b,
                        "a_changes": 0,
                        "b_changes": 0,
                        "last_a": pos_a,
                        "last_b": pos_b,
                        "run_length_a": 1,
                        "run_length_b": 1,
                    }
                    continue

                if current_segment is not None:
                    current_segment["a_pattern"] += pos_a
                    current_segment["b_pattern"] += pos_b

                    if pos_a != current_segment["last_a"]:
                        current_segment["a_changes"] += 1
                        current_segment["run_length_a"] = 1
                    else:
                        current_segment["run_length_a"] += 1

                    if pos_b != current_segment["last_b"]:
                        current_segment["b_changes"] += 1
                        current_segment["run_length_b"] = 1
                    else:
                        current_segment["run_length_b"] += 1

                    if lead != "same" and (pos_a, pos_b) not in current_segment["positions"]:
                        current_segment["positions"].append((pos_a, pos_b))

                    both_changed = (
                        pos_a != current_segment["last_a"]
                        and pos_b != current_segment["last_b"]
                    )

                    end_segment = False

                    if lead == "same":
                        end_segment = True
                    elif not both_changed and (
                        current_segment["run_length_a"] > max_run_length
                        or current_segment["run_length_b"] > max_run_length
                    ):
                        end_segment = True

                    if end_segment:
                        current_segment["end"] = pos_idx - 1
                        current_segment["length"] = (
                            current_segment["end"] - current_segment["start"] + 1
                        )

                        seg_leads = relative_leads[current_segment["start"]:pos_idx]
                        lead_changes = count_lead_changes(seg_leads)

                        if (
                            current_segment["a_changes"] >= min_changes
                            and current_segment["b_changes"] >= min_changes
                            and lead_changes >= min_lead_changes
                        ):
                            tussle_segments.append(current_segment)

                        current_segment = None

                        if lead != "same":
                            current_segment = {
                                "start": pos_idx,
                                "positions": [(pos_a, pos_b)],
                                "a_pattern": pos_a,
                                "b_pattern": pos_b,
                                "a_changes": 0,
                                "b_changes": 0,
                                "last_a": pos_a,
                                "last_b": pos_b,
                                "run_length_a": 1,
                                "run_length_b": 1,
                            }

                    if current_segment:
                        current_segment["last_a"] = pos_a
                        current_segment["last_b"] = pos_b

            if current_segment is not None:
                current_segment["end"] = min_len - 1
                current_segment["length"] = (
                    current_segment["end"] - current_segment["start"] + 1
                )

                seg_leads = relative_leads[current_segment["start"] : current_segment["end"] + 1]
                lead_changes = count_lead_changes(seg_leads)

                if (
                    current_segment["a_changes"] >= min_changes
                    and current_segment["b_changes"] >= min_changes
                    and lead_changes >= min_lead_changes
                ):
                    tussle_segments.append(current_segment)

            merged_segments = merge_tussle_segments(tussle_segments, max_gap)

            for segment in merged_segments:
                a_pattern = a_positions[segment["start"] : segment["end"] + 1]
                b_pattern = b_positions[segment["start"] : segment["end"] + 1]

                result = {
                    "driver_a": driver_a_name,
                    "driver_b": driver_b_name,
                    "start_position": segment["start"] + 1,
                    "end_position": segment["end"] + 1,
                    "driver_a_pattern": a_pattern,
                    "driver_b_pattern": b_pattern,
                    "positions_contested": format_positions(segment["positions"]),
                    "driver_a_changes": segment["a_changes"],
                    "driver_b_changes": segment["b_changes"],
                    "tussle_length": segment["length"],
                }
                results.append(result)

    return pd.DataFrame(results).sort_values(
        ["start_position", "tussle_length"], ascending=[True, False]
    ) if results else pd.DataFrame(columns=[
        "driver_a", "driver_b", "start_position", "end_position",
        "driver_a_pattern", "driver_b_pattern", "positions_contested",
        "driver_a_changes", "driver_b_changes", "tussle_length"
    ])


def count_lead_changes(lead_sequence):
    filtered = [lead for lead in lead_sequence if lead != "same"]
    if not filtered:
        return 0
    changes = 0
    last = filtered[0]
    for lead in filtered[1:]:
        if lead != last:
            changes += 1
            last = lead
    return changes


def merge_tussle_segments(segments, max_gap=1):
    if not segments or len(segments) < 2:
        return segments

    segments.sort(key=lambda x: x["start"])
    merged = []
    current = segments[0]

    for next_seg in segments[1:]:
        if next_seg["start"] <= current["end"] + max_gap + 1:
            current["end"] = next_seg["end"]
            current["length"] = current["end"] - current["start"] + 1
            current["a_changes"] += next_seg["a_changes"]
            current["b_changes"] += next_seg["b_changes"]
            for pair in next_seg["positions"]:
                if pair not in current["positions"]:
                    current["positions"].append(pair)
        else:
            merged.append(current)
            current = next_seg

    merged.append(current)
    return merged


def format_positions(position_pairs):
    if not position_pairs:
        return ""
    position_pairs.sort()
    transitions = []
    for a, b in position_pairs:
        transition = f"{a}/{b}"
        if transition not in transitions:
            transitions.append(transition)
    return f"positions {' → '.join(transitions)}" if len(transitions) > 1 else f"positions {transitions[0]}"


def analyze_patterns(driver_a_pattern, driver_b_pattern):
    total_length = len(driver_a_pattern)
    scenarios = []

    alternating_a = all(
        (driver_a_pattern[i] == driver_a_pattern[0] if i % 2 == 0 else driver_a_pattern[i] == driver_a_pattern[1])
        for i in range(min(5, total_length))
    )
    alternating_b = all(
        (driver_b_pattern[i] == driver_b_pattern[0] if i % 2 == 0 else driver_b_pattern[i] == driver_b_pattern[1])
        for i in range(min(5, total_length))
    )

    if alternating_a and alternating_b and total_length >= 4:
        scenarios.append("alternating battle")

    a_runs = [(k, len(list(g))) for k, g in groupby(driver_a_pattern)]
    b_runs = [(k, len(list(g))) for k, g in groupby(driver_b_pattern)]

    if len(a_runs) >= 3 and all(r[1] <= 2 for r in a_runs):
        scenarios.append(f"{driver_a_pattern} yo-yo pattern")
    if len(b_runs) >= 3 and all(r[1] <= 2 for r in b_runs):
        scenarios.append(f"{driver_b_pattern} yo-yo pattern")

    return scenarios


def generate_tussle_commentary(tussles_df):
    commentary = []
    for _, row in tussles_df.iterrows():
        scenarios = analyze_patterns(row["driver_a_pattern"], row["driver_b_pattern"])
        scenario_text = f" This appears to be a {scenarios[0]}." if scenarios else ""

        if row["driver_a_changes"] > row["driver_b_changes"] * 1.5:
            change_desc = f"with {row['driver_a']} being the more aggressive driver ({row['driver_a_changes']} changes vs {row['driver_b_changes']})"
        elif row["driver_b_changes"] > row["driver_a_changes"] * 1.5:
            change_desc = f"with {row['driver_b']} being the more aggressive driver ({row['driver_b_changes']} changes vs {row['driver_a_changes']})"
        else:
            change_desc = f"with both drivers making similar numbers of moves ({row['driver_a_changes']} and {row['driver_b_changes']} changes)"

        comment = (
            f"{row['driver_a']} and {row['driver_b']} were in a {row['tussle_length']}-stage tussle from "
            f"stages {row['start_position']} to {row['end_position']} contesting {row['positions_contested']} "
            f"{change_desc}.{scenario_text}"
        )
        commentary.append(comment)
    return commentary

data = {
    "driverName": ["Driver1", "Driver2", "Driver3", "Driver4", "Driver5", "Driver6"],
    "posEndingString": [
        "aaabababbb",
        "bbbababaaa",
        "cdcdcccc",
        "dcdcddeddd",
        "eeeeeedecc",
        "ffffffff",
    ],
}

df = pd.DataFrame(data)
tussles_df = detect_tussles_in_dataframe(df, min_changes=2, max_run_length=2, max_gap=1)
print("\nDetected Tussles:")
print(tussles_df)

comments = generate_tussle_commentary(tussles_df)
print("\nTussle Commentary:")
for comment in comments:
    print(comment)


Detected Tussles:
  driver_a driver_b  start_position  end_position driver_a_pattern  \
1  Driver3  Driver4               1             6           cdcdcc   
0  Driver1  Driver2               3             9          abababb   
2  Driver4  Driver5               5             9            ddedd   

  driver_b_pattern        positions_contested  driver_a_changes  \
1           dcdcdd  positions c/d → c/e → d/c                 4   
0          bababaa        positions a/b → b/a                 5   
2            eedec  positions d/c → d/e → e/d                 2   

   driver_b_changes  tussle_length  
1                 5              6  
0                 5              7  
2                 3              5  

Tussle Commentary:
Driver3 and Driver4 were in a 6-stage tussle from stages 1 to 6 contesting positions c/d → c/e → d/c with both drivers making similar numbers of moves (4 and 5 changes). This appears to be a alternating battle.
Driver1 and Driver2 were in a 7-stage tussle from st

In [7]:
from wrc_rallydj.livetiming_api import WRCLiveTimingAPIClient, time_to_seconds
from rules_processor import p, Nth
from pandas import merge, DataFrame


# The cacheing is tricky:
# - we want to be able to force updates for live stages etc
# There is internal state in the class, which stores
# the last requested data unless we force an update
# wrc = WRCLiveTimingAPIClient(use_cache=True, backend="sqlite", expire_after=600)
wrc = WRCLiveTimingAPIClient(use_cache=True, backend="memory", expire_after=600)

# wrc.initialise(year=2024)
# wrc.seasonId
# Current year, latest race by default, wrc class
wrc.initialise()

ModuleNotFoundError: No module named 'wrc_rallydj.livetiming_api'

In [4]:
wrc.stageId = "SS1"
wrc.stageId

'8330'

In [3]:
wrc.getStageDetails()
wrc.stage_codes

{'SHD': 'SHD',
 'SS1': '8330',
 'SS2': '8331',
 'SS3': '8332',
 'SS4': '8333',
 'SS5': '8334',
 'SS6': '8335',
 'SS7': '8336',
 'SS8': '8337',
 'SS9': '8338',
 'SS10': '8339',
 'SS11': '8340',
 'SS12': '8341',
 'SS13': '8342',
 'SS14': '8343',
 'SS15': '8344',
 'SS16': '8345',
 'SS17': '8346',
 'SS18': '8347',
 'FINAL': 'FINAL'}

In [5]:
stage_details = wrc.getStageDetails()
stage_details.head()

,id,stageNo,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
0,ee55daa1-fd07-574f-835a-f39008081b95,SHD,shakedown,SHD,535,Complete,,Shakedown,NaN
1,d560e801-c242-547b-baa1-881fcf11d8d1,SS1,SpecialStage,8330,535,Completed,Thursday,SS1 Umeå Sprint 1 (5.16km),5.16
2,7b3fc9fa-348b-5afd-a956-82aab0a68058,SS2,SpecialStage,8331,535,Completed,Friday,SS2 Bygdsiljum 1 (28.27km),28.27
3,4a48fad1-cd0e-59ac-b632-4613fb242f95,SS3,SpecialStage,8332,535,Completed,Friday,SS3 Andersvattnet 1 (20.51km),20.51
4,13100067-f837-5d2d-8a75-bba344f39f7f,SS4,SpecialStage,8333,535,Completed,Friday,SS4 Bäck 1 (10.8km),10.80


In [6]:
STAGE='SS3'

wrc.getOverall(stageId="SS2").head()

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,diffPrev,groupClass,championshipId,totalTimeInS,timeToCarBehind,overallGap,overallDiff,eventId,rallyId,stageId
0,a402156a-a574-503d-8992-47d081c8eba5,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom,Flags/GBR.png,...,,RC1,289,1040.4,2.7,0.0,0.0,535,583,8331
1,03d073d5-2e53-51dc-a25e-93497a622b35,2,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,+2.7,RC1,289,1043.1,3.8,2.7,2.7,535,583,8331
2,bf8b45f0-afea-5c55-8067-a1f1d7f10cfe,3,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,+3.8,RC1,289,1046.9,1.1,6.5,3.8,535,583,8331
3,03b5d275-58e9-5ff8-8fd2-fd5d25c54a23,4,#16,bf956e8a-5cad-5327-add0-26e0481ea508,France,Flags/FRA.png,Adrien FOURMAUX,8231e16a-f455-56fa-9b3c-dc3befa854cf,France,Flags/FRA.png,...,+1.1,RC1,289,1048.0,4.2,7.6,1.1,535,583,8331
4,4f1dc222-a7ba-5afd-bce7-12733bb7e86d,5,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,+4.2,RC1,289,1052.2,2.1,11.8,4.2,535,583,8331


In [4]:
wrc.getStageTimes(stageId=STAGE).head()

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,championshipId,eventId,rallyId,Gap,Diff,timeInS,timeToCarBehind,speed (km/h),pace (s/km),pace diff (s/km)
0,d71b21a3-0e54-502c-8aa6-0ef5fbd8b153,1,#16,bf956e8a-5cad-5327-add0-26e0481ea508,France,Flags/FRA.png,Adrien FOURMAUX,8231e16a-f455-56fa-9b3c-dc3befa854cf,France,Flags/FRA.png,...,289,535,583,0.0,0.0,643.8,-1.8,114.687791,31.389566,0.000000
1,ffc62051-dc3e-5fae-80a5-506b2581d64f,2,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom,Flags/GBR.png,...,289,535,583,1.8,1.8,645.6,-1.2,114.368030,31.477328,0.087762
2,a5e946b4-41d7-5606-8f4e-03a9aefa3d10,3,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,289,535,583,3.0,1.2,646.8,0.0,114.155844,31.535836,0.146270
3,3e80424e-26d5-52a9-a38b-b49818ec889e,4,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,289,535,583,3.0,0.0,646.8,-1.7,114.155844,31.535836,0.146270
4,8a654f4a-fd8d-55ff-9d0c-8df626a8d62f,5,#1,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,...,289,535,583,4.7,1.7,648.5,-2.2,113.856592,31.618723,0.229157


In [5]:
# Previous
prev_idx = stage_details.loc[stage_details["stageNo"]==STAGE].index[0]
PREV_STAGE = stage_details.loc[prev_idx-1, "stageNo"] if STAGE!="SS1" and STAGE!="SHD" else ""
PREV_STAGE

'SS2'

In [ ]:
def core_stage(stageNo):
    retcols_stage = ["carNo", "driver", "pos", "Gap", "Diff"]
    retcols_overall = ["carNo", "driver", "pos", "overallGap", "overallDiff", "timeToCarBehind"]

    prev_idx = stage_details.loc[stage_details["stageNo"] == stageNo].index[0]
    prevStageNo = (
        stage_details.loc[prev_idx - 1, "stageNo"]
        if STAGE != "SS1" and STAGE != "SHD"
        else ""
    )
    # print(prevStageNo, stageNo)
    _df_stage_curr = wrc.getStageTimes(stageId=stageNo)[retcols_stage].copy()

    _df_overall_curr = wrc.getOverall(stageId=stageNo, update=True)[
        retcols_overall
    ].copy()
    _df_overall_curr.rename(columns={"pos": "overallPos"}, inplace=True)
    _df_overall_prev = wrc.getOverall(stageId=prevStageNo, update=True)[
        retcols_overall
    ].copy()
    _df_overall_prev.rename(columns={"pos": "overallPos"}, inplace=True)
    # The subtraction is this way to handle signs better
    display(_df_overall_prev)
    display(_df_overall_curr)
    _df_overall_diff = (
        _df_overall_prev.set_index(["carNo", "driver"])
        - _df_overall_curr.set_index(["carNo", "driver"])
    ).reset_index()
    _df_overall_diff.rename(
        columns={
            "overallPos": "overallPosDelta",
            "overallGap": "overallGapDelta",
            "overallDiff": "overallDiffDelta",
            "timeToCarBehind": "timeToCarBehindDelta",
        },
        inplace=True,
    )
    _df_overall_curr["currPodium"] = _df_overall_curr["overallPos"] <= 3
    _df_overall = merge(_df_overall_diff, _df_overall_curr, on=["carNo", "driver"])

    _df_overall_prev["prevLeader"] = _df_overall_prev["overallPos"] == 1
    _df_overall_prev["prevOverallPos"] = _df_overall_prev["overallPos"]
    _df_overall_prev["prevPodium"] = _df_overall_prev["overallPos"] <= 3
    _df_overall = merge(
        _df_overall,
        _df_overall_prev[["carNo", "driver", "prevLeader", "prevOverallPos", "prevPodium"]],
        on=["carNo", "driver"],
    )

    _df_overall["overallPosChange"] = _df_overall["overallPosDelta"]!=0
    _df_overall["currLeader"] = _df_overall["overallPos"] == 1
    _df_overall["onto_podium"] = (~_df_overall["prevPodium"]) & _df_overall["currPodium"]
    _df_overall["lost_podium"] = (_df_overall["prevPodium"]) & ~_df_overall["currPodium"]
    _df_overall["newLeader"] = (
        _df_overall["overallPosChange"] & _df_overall["currLeader"]
    )
    _df_overall["lead_changed"] = _df_overall["newLeader"].any()

    _df_overall = merge( _df_overall, _df_stage_curr,
        on=["carNo", "driver"],
    )
    
    return _df_overall

_overall_diff= core_stage("SS6")

display(_overall_diff)
display(_stage)

,carNo,driver,overallPos,overallGap,overallDiff,timeToCarBehind
0,#18,Takamoto KATSUTA,1,0.0,0.0,5.8
1,#33,Elfyn EVANS,2,5.8,5.8,3.9
2,#8,Ott TÄNAK,3,9.7,3.9,0.2
3,#16,Adrien FOURMAUX,4,9.9,0.2,3.5
4,#1,Thierry NEUVILLE,5,13.4,3.5,11.5
5,#69,Kalle ROVANPERÄ,6,24.9,11.5,25.7
6,#55,Joshua MCERLEAN,7,50.6,25.7,11.6
7,#5,Sami PAJARI,8,62.2,11.6,6.9
8,#13,Grégoire MUNSTER,9,69.1,6.9,NaN


,carNo,driver,overallPos,overallGap,overallDiff,timeToCarBehind
0,#33,Elfyn EVANS,1,0.0,0.0,1.9
1,#18,Takamoto KATSUTA,2,1.9,1.9,1.7
2,#8,Ott TÄNAK,3,3.6,1.7,5.7
3,#16,Adrien FOURMAUX,4,9.3,5.7,5.0
4,#1,Thierry NEUVILLE,5,14.3,5.0,11.8
5,#69,Kalle ROVANPERÄ,6,26.1,11.8,24.1
6,#55,Joshua MCERLEAN,7,50.2,24.1,20.7
7,#5,Sami PAJARI,8,70.9,20.7,2.5
8,#13,Grégoire MUNSTER,9,73.4,2.5,NaN


,carNo,driver,overallPosDelta,overallGapDelta,overallDiffDelta,timeToCarBehindDelta,overallPos,overallGap,overallDiff,timeToCarBehind,...,prevPodium,overallPosChange,currLeader,onto_podium,lost_podium,newLeader,lead_changed,pos,Gap,Diff
0,#1,Thierry NEUVILLE,0,-0.9,-1.5,-0.3,5,14.3,5.0,11.8,...,False,False,False,False,False,False,True,5,7.0,1.3
1,#13,Grégoire MUNSTER,0,-4.3,4.4,NaN,9,73.4,2.5,NaN,...,False,False,False,False,False,False,True,8,10.4,2.4
2,#16,Adrien FOURMAUX,0,0.6,-5.5,-1.5,4,9.3,5.7,5.0,...,False,False,False,False,False,False,True,3,5.5,5.2
3,#18,Takamoto KATSUTA,-1,-1.9,-1.9,4.1,2,1.9,1.9,1.7,...,True,True,False,False,False,False,True,7,8.0,0.7
4,#33,Elfyn EVANS,1,5.8,5.8,2.0,1,0.0,0.0,1.9,...,True,True,True,False,False,True,True,2,0.3,0.3
5,#5,Sami PAJARI,0,-8.7,-9.1,4.4,8,70.9,20.7,2.5,...,False,False,False,False,False,False,True,9,14.8,4.4
6,#55,Joshua MCERLEAN,0,0.4,1.6,-9.1,7,50.2,24.1,20.7,...,False,False,False,False,False,False,True,4,5.7,0.2
7,#69,Kalle ROVANPERÄ,0,-1.2,-0.3,1.6,6,26.1,11.8,24.1,...,False,False,False,False,False,False,True,6,7.3,0.3
8,#8,Ott TÄNAK,0,6.1,2.2,-5.5,3,3.6,1.7,5.7,...,True,False,False,False,False,False,True,1,0.0,0.0


,carNo,driver,pos,Gap,Diff
0,#8,Ott TÄNAK,1,0.0,0.0
1,#33,Elfyn EVANS,2,0.3,0.3
2,#16,Adrien FOURMAUX,3,5.5,5.2
3,#55,Joshua MCERLEAN,4,5.7,0.2
4,#1,Thierry NEUVILLE,5,7.0,1.3
5,#69,Kalle ROVANPERÄ,6,7.3,0.3
6,#18,Takamoto KATSUTA,7,8.0,0.7
7,#13,Grégoire MUNSTER,8,10.4,2.4
8,#5,Sami PAJARI,9,14.8,4.4


In [48]:
_overall_diff.columns

Index(['carNo', 'driver', 'overallPosDelta', 'overallGapDelta',
       'overallDiffDelta', 'timeToCarBehindDelta', 'overallPos', 'overallGap',
       'overallDiff', 'timeToCarBehind', 'currPodium', 'prevLeader',
       'prevOverallPos', 'prevPodium', 'overallPosChange', 'currLeader',
       'onto_podium', 'lost_podium', 'newLeader', 'lead_changed', 'pos', 'Gap',
       'Diff'],
      dtype='object')

In [49]:
leader_row = _overall_diff[_overall_diff["overallPos"] == 1].iloc[0]
leader_row

carNo                           #33
driver                  Elfyn EVANS
overallPosDelta                   1
overallGapDelta                 5.8
overallDiffDelta                5.8
timeToCarBehindDelta            2.0
overallPos                        1
overallGap                      0.0
overallDiff                     0.0
timeToCarBehind                 1.9
currPodium                     True
prevLeader                    False
prevOverallPos                    2
prevPodium                     True
overallPosChange               True
currLeader                     True
onto_podium                   False
lost_podium                   False
newLeader                      True
lead_changed                   True
pos                               2
Gap                             0.3
Diff                            0.3
Name: 4, dtype: object

In [51]:
def rule_onto_podium(row):
    remark = ""
    if row["onto_podium"]:
        remark = f"""{row["driver"]} moved into a podium position"""
    return (remark, 0.9)

def rule_into_first(row):
    remark = ""
    if row["newLeader"]:
        big_jump = f"""jumped { p.number_to_words(row["overallPosDelta"])} places""" if row["overallPosDelta"]>1 else "moved"
        remark = f"""Taking {Nth(row["pos"])} place on stage, {row["driver"]} {big_jump} into first place overall, taking a lead of {round(row["timeToCarBehind"], 1)}s"""
    return (remark, 1.0)

def rule_lost_first(row):
    remark = ""
    if row["prevLeader"] and not row["currLeader"]:
        remark = f"""Coming in at {Nth(row["pos"])} on the stage, {row["Gap"]}s behind the stage winner, {row["driver"]} lost the overall lead, falling back to {Nth(row["overallPos"])} place"""

        if row["overallPos"]>=2:
            fell_back = f"""{remark}, {row["overallGap"]}s behind the new leader."""
        else:
            remark = f"{remark}."

    return (remark, 0.9)


# TO DO - need a natural time for timeInS

def rule_leader_increased_lead(row):
    remark = ""
    if row["overallGapDelta"] > 0 and row["currLeader"] and not row["newLeader"]:
        remark = f"""{row["driver"]} increased his the lead at the front of the rally, moving a furher {row["overallGapDelta"]}s ahead, to {row["overallGapDelta"]}s."""

    return (remark, 0.7)


def rule_leader_decreased_lead(row):
    remark = ""
    if row["overallGapDelta"] < 0 and row["currLeader"] and not row["newLeader"]:
        remark = f"""At the front, {row["driver"]}'s lead was reduced by   {row["overallGapDelta"]}s to {row["overallGapDelta"]}s."""

    return (remark, 0.7)

def rule_onto_podium(row):
    if row["onto_podium"]:
        remark = ""


def process_rules(df):
    # Apply each rule to create new columns
    remarks_df = DataFrame(
        {
            "podium_remarks": df.apply(rule_onto_podium, axis=1),
            "into_first_remarks": df.apply(rule_into_first, axis=1),
            "lost_first_remarks": df.apply(rule_lost_first, axis=1),
            "lead_increased_remarks": df.apply(rule_leader_increased_lead, axis=1),
            "lead_reduced_remarks": df.apply(rule_leader_decreased_lead, axis=1),
            "move_into_podium_remarks": df.apply(rule_onto_podium, axis=1),
        }
    )

    # Stack all remarks into a single series and filter out empty strings
    filtered_remarks = [(remark[0].replace("  "," "), remark[1]) for remark in remarks_df.stack() if remark[0] != ""]

    return filtered_remarks

remarks = process_rules(_overall_diff)
remarks

[('Coming in at seventh on the stage, 8.0s behind the stage winner, Takamoto KATSUTA lost the overall lead, falling back to second',
  0.9),
 ('Elfyn EVANS moved into first place overall, taking a lead of 1.9s', 1.0),
 ('.', 0.9)]

In [17]:
itinerary_df.head()

,stage,eventId,stageId,distance,timingPrecision,firstCarDueDateTime,firstCarDueDateTimeMs,controlPenalties,status,type,location,targetDuration,targetDurationMs,id,order,date
0,TC0,535,null,,Minute,18:40,2025-02-13T18:40:00+01:00,All,Completed,TimeControl,Podium Red Barn Arena,null,null,1506.0,1.0,Thursday 13th February
1,TC1,535,d560e801-c242-547b-baa1-881fcf11d8d1,4.97 km,Minute,19:00,2025-02-13T19:00:00+01:00,All,Completed,TimeControl,Umeå Sprint,00:20:00,1200000,1506.0,1.0,Thursday 13th February
2,SS1,535,d560e801-c242-547b-baa1-881fcf11d8d1,5.16 km,Minute,19:05,2025-02-13T19:05:00+01:00,None,Completed,StageStart,Umeå Sprint 1,00:05:00,300000,1506.0,1.0,Thursday 13th February
3,SF1,535,d560e801-c242-547b-baa1-881fcf11d8d1,,Tenth,,,None,Completed,FlyingFinish,Umeå Sprint 1,null,null,1506.0,1.0,Thursday 13th February
4,TC1A,535,null,2.87 km,Minute,19:35,2025-02-13T19:35:00+01:00,Late,Completed,TimeControl,Parc Fermé Umeå IN,00:30:00,1800000,1506.0,1.0,Thursday 13th February


## Itinerary reports

TO DO:

- use similar approach to above;
- split into "pre_stage" and "post_stage" components
- maybe we should in general return a dict?
- make more use of itinerary as a data source/enrich itinerary/replace stage_info?
- start to explore scope for stage route analysis remarks (eg can we indetify and count hairpins? Junctions?)

- do a "heads up for the day" analysis from the itinerary, summarising what will happen over the course of the day

In [ ]:
from datetime import datetime
import re

itinerary_df = wrc.getItinerary(latest=True)
md = []
stage_code="SS4"
wrc.stageId = stage_code

# TO DO - we can replace the stage_info_row data with
# enriched itinerary data
stage_info = wrc.getStageDetails(update=True)
stage_info["stageInDay"] = stage_info.groupby(["day"]).cumcount() + 1
ss_index = itinerary_df[itinerary_df["stage"] == stage_code].index[0]

stage_info_row = stage_info.loc[
                            stage_info["stageNo"] == stage_code
                        ]
stage_name = stage_info_row.iloc[0]["name"]

_md = stage_name
# Remark on, or imply, the repeated run nature of this stage
repeated_run = re.match(r".*\s(\d+)\s+\(.*", stage_name)
if repeated_run:
    _md = f"{_md}, the {Nth(int(repeated_run.group(1)))} run of this stage"

# Remark on being the Nth stage of the day
_md = f"""{_md}, the {Nth(stage_info_row.iloc[0]["stageInDay"])}"""
if (
    stage_info_row.iloc[0]["stageInDay"]
    == stage_info[
        stage_info["day"] == stage_info_row.iloc[0]["day"]
    ]["stageInDay"].max()
):
    _md = f"{_md}, and last,"
start_time = datetime.fromisoformat(
    itinerary_df.iloc[ss_index]["firstCarDueDateTimeMs"]
)
time_str = (
    start_time.strftime("starting at %I.%M%p")
    .lower()
    .replace(" 0", " ")
)

_md = f"""{_md} stage of the day {stage_code} ({stage_info_row.iloc[0]["day"]}), {time_str}."""

# Remark on being the longest stage of the rally
if (
    stage_info_row.iloc[0]["distance"]
    == stage_info["distance"].max()
):
    _md = f"{_md} It is the longest stage on the rally."
md.append(f"{_md}\n\n")


# Previous liaison - do this as enrichment?
previous_tc = itinerary_df.iloc[ss_index - 1]
previous_out = itinerary_df.iloc[ss_index - 2]
previous_location = (
    f"previous {previous_out['location']} stage"
    if previous_out["type"] == "FlyingFinish"
    else f'{previous_out["location"]} {previous_out["type"]}'
)
art_ = p.a(
    p.number_to_words(
        float(previous_tc["distance"].split()[0])
    )
).split()[0]
_md = f'Prior to the stage, {art_} {previous_tc["distance"]} liaison section to the {previous_tc["location"]} {previous_tc["type"]} from the {previous_location}.'
md.append(_md)

# End of stage
future_ = itinerary_df.iloc[ss_index + 1 :]
# Get indices of time controls
next_tc_idx = future_[
    future_["stage"].str.startswith("T")
].index[0]
next_tc = itinerary_df.iloc[next_tc_idx]
art_ = p.a(
    p.number_to_words(float(next_tc["distance"].split()[0]))
).split()[0]
arrival_time = datetime.fromisoformat(
    next_tc["firstCarDueDateTimeMs"]
)
next_arrival_time = (
    arrival_time.strftime("from %I.%M%p")
    .lower()
    .replace(" 0", " ")
)
_md_final = f'Following the stage, {art_} {next_tc["distance"]} liaison section to {next_tc["location"]} (stage running from {next_arrival_time}).'

# Stage status
state_status = itinerary_df.iloc[ss_index]["status"]
if state_status:
    md.append(f"Stage status: *{state_status}*.")

md

['SS4 Bäck 1 (10.8km), the first run of this stage, the third stage of the day SS4 (Friday), starting at 11.27am.\n\n',
 'Prior to the stage, a 41.01 km liaison section to the Bäck TimeControl from the previous Andersvattnet 1 stage.',
 'Stage status: *Completed*.']

In [6]:
itinerary_df.head()
# TO DO - we need to merge in an appropriate stageId from eg wrc.getStageDetails()

,stage,eventId,stageId,distance,timingPrecision,firstCarDueDateTime,firstCarDueDateTimeMs,controlPenalties,status,type,location,targetDuration,targetDurationMs,id,order,date
0,TC0,535,null,,Minute,18:40,2025-02-13T18:40:00+01:00,All,Completed,TimeControl,Podium Red Barn Arena,null,null,1506.0,1.0,Thursday 13th February
1,TC1,535,d560e801-c242-547b-baa1-881fcf11d8d1,4.97 km,Minute,19:00,2025-02-13T19:00:00+01:00,All,Completed,TimeControl,Umeå Sprint,00:20:00,1200000,1506.0,1.0,Thursday 13th February
2,SS1,535,d560e801-c242-547b-baa1-881fcf11d8d1,5.16 km,Minute,19:05,2025-02-13T19:05:00+01:00,None,Completed,StageStart,Umeå Sprint 1,00:05:00,300000,1506.0,1.0,Thursday 13th February
3,SF1,535,d560e801-c242-547b-baa1-881fcf11d8d1,,Tenth,,,None,Completed,FlyingFinish,Umeå Sprint 1,null,null,1506.0,1.0,Thursday 13th February
4,TC1A,535,null,2.87 km,Minute,19:35,2025-02-13T19:35:00+01:00,Late,Completed,TimeControl,Parc Fermé Umeå IN,00:30:00,1800000,1506.0,1.0,Thursday 13th February


## Stage Report

In [7]:
# Need to set a stage
#wrc.stageId = "SS3"
#stage_info=

In [9]:
wrc.stage_ids

{'SHD': 'SHD',
 '8330': 'SS1',
 '8331': 'SS2',
 '8332': 'SS3',
 '8333': 'SS4',
 '8334': 'SS5',
 '8335': 'SS6',
 '8336': 'SS7',
 '8337': 'SS8',
 '8338': 'SS9',
 '8339': 'SS10',
 '8340': 'SS11',
 '8341': 'SS12',
 '8342': 'SS13',
 '8343': 'SS14',
 '8344': 'SS15',
 '8345': 'SS16',
 '8346': 'SS17',
 '8347': 'SS18',
 'FINAL': 'FINAL'}

In [ ]:
from wrc_rallydj.livetiming_api import enrich_stage_winners

times = wrc.getStageTimes()
overall_df = wrc.getOverall()

md = []

overall_pos = overall_df.loc[
    overall_df["carNo"] == times.iloc[0]["carNo"], "pos"
].iloc[0]

_md = f"""{times.iloc[0]["driver"]} was in {Nth(1)} position on the stage and {Nth(overall_pos)} overall.
"""
md.append(_md)

stagewinners = wrc.getStageWinners()
stages = wrc.getStageDetails()

stagewinners = enrich_stage_winners(stagewinners, stages)

if not stagewinners.empty:
    winner_row = stagewinners.loc[stagewinners["stageNo"] == wrc.stage_ids[stage_code]]

    _md = f"""This was his {Nth(winner_row.iloc[0]["daily_wins"])} stage win of the day and his {Nth(winner_row.iloc[0]["wins_overall"])} stage win overall."""

    md.append(_md)

if times.iloc[0]["carNo"] != overall_df.iloc[0]["carNo"]:
    leader_row = times.loc[
        times["carNo"] == overall_df.iloc[0]["carNo"]
    ]
    leader = leader_row.iloc[0]["driver"]
else:
    leader = ""
    leader_row = DataFrame()

CLOSE_PACE = 0.1  # 0.05
on_the_pace = times[times["pace diff (s/km)"] < CLOSE_PACE]
leader_handled = False
if len(on_the_pace) > 1:
    _md = "Also on the pace"
    for _, r in on_the_pace[1:].iterrows():
        if leader == r["driver"]:
            leader_handled = True
            leader_text = "rally leader "
        else:
            leader_text = ""
        _md = (
            _md
            + f""", {leader_text}{r["driver"]} was just {r["diffFirst"]}s behind ({round(r["pace diff (s/km)"], 2)} s/km off the stage winner)"""
        )
    md.append(_md + ".")

if (
    not leader_row.empty and not leader_handled
):  # Check if leader exists in times
    leaderPos = leader_row.iloc[0]["pos"]
    leaderDiff = leader_row.iloc[0]["diffPrev"]
    _md = f"""Rally leader {overall_df.iloc[0]["driver"]} was {leaderDiff} seconds behind in {Nth(leaderPos)} position."""
    md.append(_md)  # Properly append the string

md

AttributeError: 'function' object has no attribute 'stage'

In [5]:
wrc.getStageWinners()

,id,carNo,stageNo,stageName,stageType,stageId,eventId,entryId,driverId,driverCountry,...,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,time,timeInS
0,535,#33,SS1,Umeå Sprint 1 (5.16 km),SpecialStage,8330,be2793cd-d2af-5b68-be24-da064353ecd4,1860fe9a-a2bf-5d1c-bc03-139e4d0235dc,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,...,United Kingdom,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1,Toyota,teamLogo/toyota.png,M,3:21.6,201.6
1,535,#33,SS2,Bygdsiljum 1 (28.27 km),SpecialStage,8331,be2793cd-d2af-5b68-be24-da064353ecd4,1860fe9a-a2bf-5d1c-bc03-139e4d0235dc,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,...,United Kingdom,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1,Toyota,teamLogo/toyota.png,M,13:58.8,838.8
2,535,#16,SS3,Andersvattnet 1 (20.51 km),SpecialStage,8332,be2793cd-d2af-5b68-be24-da064353ecd4,5a515fdf-738f-54ae-89ff-57aef9e31007,bf956e8a-5cad-5327-add0-26e0481ea508,France,...,France,Flags/FRA.png,Alexandre CORIA,b6692ea5-df92-5cad-a91c-20319a6fffd7,i20 N Rally1,Hyundai,teamLogo/hyundai.png,M,10:43.8,643.8
3,535,#16,SS4,Bäck 1 (10.8 km),SpecialStage,8333,be2793cd-d2af-5b68-be24-da064353ecd4,5a515fdf-738f-54ae-89ff-57aef9e31007,bf956e8a-5cad-5327-add0-26e0481ea508,France,...,France,Flags/FRA.png,Alexandre CORIA,b6692ea5-df92-5cad-a91c-20319a6fffd7,i20 N Rally1,Hyundai,teamLogo/hyundai.png,M,5:54.6,354.6
4,535,#18,SS5,Bygdsiljum 2 (28.27 km),SpecialStage,8334,be2793cd-d2af-5b68-be24-da064353ecd4,4d07cf1f-18a4-572e-8ec3-5db956725e37,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,...,Ireland,Flags/IRL.png,Aaron JOHNSTON,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1,Toyota,teamLogo/toyota.png,M,13:50.1,830.1
5,535,#8,SS6,Andersvattnet 2 (20.51 km),SpecialStage,8335,be2793cd-d2af-5b68-be24-da064353ecd4,f92ab26e-23eb-59ad-b938-ec443ed8579a,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,...,Estonia,Flags/EST.png,Martin JÄRVEOJA,b6692ea5-df92-5cad-a91c-20319a6fffd7,i20 N Rally1,Hyundai,teamLogo/hyundai.png,M,10:49.2,649.2
6,535,#1,SS7,Bäck 2 (10.8 km),SpecialStage,8336,be2793cd-d2af-5b68-be24-da064353ecd4,8b9cc07a-cae7-50f5-af4b-aa627335f2af,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,...,Belgium,Flags/BEL.png,Martijn WYDAEGHE,b6692ea5-df92-5cad-a91c-20319a6fffd7,i20 N Rally1,Hyundai,teamLogo/hyundai.png,M,6:02.5,362.5
7,535,#33,SS8,Umeå Sprint 2 (5.16 km),SuperSpecialStage,8337,be2793cd-d2af-5b68-be24-da064353ecd4,1860fe9a-a2bf-5d1c-bc03-139e4d0235dc,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,...,United Kingdom,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1,Toyota,teamLogo/toyota.png,M,3:29.9,209.9
8,535,#69,SS9,Vännäs 1 (15.65 km),SpecialStage,8338,be2793cd-d2af-5b68-be24-da064353ecd4,397b6b74-9a21-5512-a479-dc17e4384b89,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,...,Finland,Flags/FIN.png,Jonne HALTTUNEN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1,Toyota,teamLogo/toyota.png,M,9:00.5,540.5
9,535,#33,SS10,Sarsjöliden 1 (14.23 km),SpecialStage,8339,be2793cd-d2af-5b68-be24-da064353ecd4,1860fe9a-a2bf-5d1c-bc03-139e4d0235dc,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,...,United Kingdom,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1,Toyota,teamLogo/toyota.png,M,6:37.5,397.5


In [6]:
wrc.getStageTimes()

""


In [7]:
wrc.eventId, wrc.rallyId, wrc.stageId, wrc.championshipId, wrc.championship

('535', '583', 'SS3', 289, 'wrc')